In [1]:
# 1. individual info: voter_id, name, birthday, party code
# 2. multiindex dataframe moving over time: 1. voter id, 2. date address acquired, 3. address, 4. lat long
# new csv file with each unique voter ID in each row and address over time

import pandas as pd
import glob
import numpy as np

In [2]:
# grab the summary data
df = pd.read_csv('../Multnomah_VoterHistory_2017-03-24/00_Ex-VotingHistory-nkramer-2017-03-21-050310.txt', sep='\t', encoding = "ISO-8859-1")

/Users/leey/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,6,14,15,20,26,27,34,35,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# df.STATUS.unique()
df.columns

Index(['VOTER_ID', 'FIRST_NAME', 'MIDDLE_NAME', 'LAST_NAME', 'NAME_SUFFIX',
       'BIRTH_DATE', 'CONFIDENTIAL', 'EFF_REGN_DATE', 'STATUS', 'PARTY_CODE',
       'PHONE_NUM', 'UNLISTED', 'COUNTY', 'RES_ADDRESS_1', 'RES_ADDRESS_2',
       'HOUSE_NUM', 'HOUSE_SUFFIX', 'PRE_DIRECTION', 'STREET_NAME',
       'STREET_TYPE', 'POST_DIRECTION', 'UNIT_TYPE', 'UNIT_NUM',
       'ADDR_NON_STD', 'CITY', 'STATE', 'ZIP_CODE', 'ZIP_PLUS_FOUR',
       'EFF_ADDRESS_1', 'EFF_ADDRESS_2', 'EFF_ADDRESS_3', 'EFF_ADDRESS_4',
       'EFF_CITY', 'EFF_STATE', 'EFF_ZIP_CODE', 'EFF_ZIP_PLUS_FOUR',
       'ABSENTEE_TYPE', 'PRECINCT_NAME', 'PRECINCT', 'SPLIT', '11/08/2016',
       '05/17/2016', '11/03/2015', '05/19/2015', '11/04/2014', '05/20/2014',
       '11/05/2013', '05/21/2013', '11/06/2012', '05/15/2012', '01/31/2012',
       '11/08/2011', '05/17/2011', '11/02/2010', '05/18/2010', '01/26/2010',
       '05/19/2009', '11/04/2008', '05/20/2008', '11/06/2007', '05/15/2007',
       '11/07/2006', '09/19/2006', '05/1

In [4]:
# pull out dates
cnames = df.columns
dates = [name for name in cnames if '/' in name]

In [5]:
idx = df[dates]!='-'
idx.astype(int).sum(axis=1)
df = df[idx.astype(int).sum(axis=1)>0]

In [ ]:
# get the list of text files
datef = glob.glob('2*txt')

In [ ]:
df.head()

In [6]:
# drop duplicates (duplicated voter ID rows are filled with 'ACP')
df = df.drop_duplicates(subset='VOTER_ID', keep=False)

# grab their voter id and birthday
voter_info = df[['VOTER_ID', 'FIRST_NAME', 'MIDDLE_NAME', 'LAST_NAME','BIRTH_DATE','PARTY_CODE']]
voter_info = voter_info.rename(columns={'VOTER_ID':'temp'})
voter_info[:3]

,temp,FIRST_NAME,MIDDLE_NAME,LAST_NAME,BIRTH_DATE,PARTY_CODE
1,519109,DANE,WILLARD,WAHLSTROM,07-20-1979,DEM
6,100064977,JAMES,EUGENE,MORTENSEN,08-07-1977,REP
7,100042592,JARED,BANKES,HERZOG,07-25-1978,DEM


In [16]:
voter_info[voter_info.VOTER_ID==300427492]

,VOTER_ID,FIRST_NAME,MIDDLE_NAME,LAST_NAME,BIRTH_DATE,PARTY_CODE
402595,300427492,SEAN,ZACHARIAH,CARLSON,08-04-1997,DEM


In [13]:
# not all voter ids are integers
voter_info['VOTER_ID'] = pd.to_numeric(voter_info.temp, downcast='integer', errors='coerce')
voter_info = voter_info[pd.notnull(voter_info['VOTER_ID'])]
voter_info = voter_info[['VOTER_ID','FIRST_NAME','MIDDLE_NAME','LAST_NAME','BIRTH_DATE','PARTY_CODE']]

In [14]:
# 1. individual info: voter_id, name, birthday, party code
voter_info[:3]

,VOTER_ID,FIRST_NAME,MIDDLE_NAME,LAST_NAME,BIRTH_DATE,PARTY_CODE
1,519109,DANE,WILLARD,WAHLSTROM,07-20-1979,DEM
6,100064977,JAMES,EUGENE,MORTENSEN,08-07-1977,REP
7,100042592,JARED,BANKES,HERZOG,07-25-1978,DEM


In [ ]:
# 1a. convert from birth date to age
voter_info['year'] = voter_info.BIRTH_DATE.apply(lambda x: x.split('-')[2] if '-' in x else np.nan)
voter_info['temp'] = pd.to_numeric(voter_info.year, downcast='integer', errors='coerce')
# get rid of NaNs, remove people with birthday year of 1111, XXXXXXXX
voter_info = voter_info.dropna(subset=['temp'])
# voter_info['age2018'] = 2018-voter_info.temp
# voter_info['age2016'] = 2016-voter_info.temp
# voter_info['age2006'] = 2006-voter_info.temp
voter_info = voter_info.drop(['temp'], axis=1)
# get rid of people over the age of 100 (removing people with birthdays in the year of 1111)
voter_info = voter_info[voter_info.age2018<100]

In [ ]:
voter_info.head()

In [ ]:
# 2. multiindex dataframe moving over time: 1. voter id, 2. date address acquired, 3. address, 4. lat long
addresses = voter_info[['VOTER_ID']]

In [ ]:
from ipywidgets import FloatProgress
from IPython.display import display

In [ ]:
max_count = len(datef)

fb = FloatProgress(min=0, max=max_count) # instantiate the bar
display(fb) # display the bar

# fill in the dictionary
for f in datef:
    # signal to increment the progress bar
    fb.value += 1
    
    # load data
    tempdf = pd.read_csv(f, sep='\t', encoding="ISO-8859-1")
    
    # drop duplicates (duplicated voter ID rows are filled with 'ACP')
    tempdf = tempdf.drop_duplicates(subset='VOTER_ID', keep=False)
    
    # not all are integers and there is one weird 'ACP' value
    tempdf = tempdf.rename(columns={'VOTER_ID':'temp'})
    tempdf['VOTER_ID'] = pd.to_numeric(tempdf.temp, downcast='integer', errors='coerce')
    tempdf = tempdf[pd.notnull(tempdf['VOTER_ID'])]
    tempdf = tempdf[pd.notnull(tempdf['RES_ADDRESS_1'])]
    
    tempdf = tempdf[['VOTER_ID', 'RES_ADDRESS_1', 'RES_ADDRESS_2']]
    
    # replace all nans in the address column
    tempdf = tempdf.replace(np.nan, '', regex=True)
    
    # concatenate on tempdf
    tempdf[f[:-4]] = tempdf[['RES_ADDRESS_1', 'RES_ADDRESS_2']].apply(lambda x: ' '.join(x), axis=1)
    
    # drop  columns
    tempdf = tempdf.drop(['RES_ADDRESS_1', 'RES_ADDRESS_2'], axis=1)
    
    # merge to the address df
    addresses = addresses.join(tempdf.set_index('VOTER_ID'), on='VOTER_ID')

In [ ]:
# dump into a csv
addresses.to_csv('voter_address_over_time.csv', sep='\t', mode = 'w', index=False)

In [ ]:
# load addresses
addresses = pd.read_csv('voter_address_over_time.csv', sep='\t')

In [ ]:
voterlatlong = pd.DataFrame(data=np.repeat(addresses.VOTER_ID, addresses.columns.shape[0] - 1), columns=['VOTER_ID'])
voterlatlong['dates'] = np.tile(addresses.columns[1:], addresses.VOTER_ID.shape[0])
voterlatlong['addresses'] = np.repeat(np.nan, voterlatlong.shape[0])

In [ ]:
voterlatlong = voterlatlong.set_index(['VOTER_ID', 'dates'])

In [ ]:
# voterlatlong['addresses']=voterlatlong['addresses'].map(addressdict, na_action='ignore')
voterlatlong.shape

In [ ]:
max_count = len(datef)

fb = FloatProgress(min=0, max=max_count) # instantiate the bar
display(fb) # display the bar

# fill in the dictionary
for f in datef:
    # signal to increment the progress bar
    fb.value += 1
    
    # load data
    tempdf = pd.read_csv(f, sep='\t', encoding="ISO-8859-1")
    
    # drop duplicates (duplicated voter ID rows are filled with 'ACP')
    tempdf = tempdf.drop_duplicates(subset='VOTER_ID', keep=False)
    
    # not all are integers and there is one weird 'ACP' value
    tempdf = tempdf.rename(columns={'VOTER_ID':'temp'})
    tempdf['VOTER_ID'] = pd.to_numeric(tempdf.temp, downcast='integer', errors='coerce')
    tempdf = tempdf[pd.notnull(tempdf['VOTER_ID'])]
    tempdf = tempdf[pd.notnull(tempdf['RES_ADDRESS_1'])]
    
    # replace all nans in the address column
    tempdf = tempdf.replace(np.nan, '', regex=True)
    
    # concatenate to full address
    tempdf['addresses'] = tempdf[['RES_ADDRESS_1', 'RES_ADDRESS_2']].apply(lambda x: ' '.join(x), axis=1)

    # set same index for tempdf and update
    tempdf['dates'] = np.repeat(f[:-4], tempdf.VOTER_ID.shape[0])
    tempdf = tempdf[['VOTER_ID', 'dates', 'addresses']]
    tempdf = tempdf.set_index(['VOTER_ID', 'dates'])
    voterlatlong.update(tempdf)

In [ ]:
voterlatlong.shape

In [ ]:
latlongs = pd.read_csv('voter_address_over_time-geocoded.csv')

In [ ]:
# now add in the lat longs
latlongs = latlongs.rename(columns={'Column1': 'addresses'})
latlongs.head()

In [ ]:
# reset index
voterlatlong = voterlatlong.reset_index()

In [ ]:
# join on address
print(voterlatlong.shape)
voterlatlong = voterlatlong.merge(latlongs, how='left', on=['addresses'], sort=False)
print(voterlatlong.shape)

In [ ]:
# write lat longs to csv
voterlatlong.to_csv('voter_address_lat_long.csv', sep='\t', mode = 'w', index=False)

In [2]:
# read voterlatlong
voterlatlong = pd.read_csv('voter_address_lat_long.csv', sep='\t')

/Users/leey/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
voterlatlong.head()

In [ ]:
voterlatlong.Lat[5]

In [ ]:
# count the number of addresses
addresses['counts'] = addresses.apply(lambda x: x.nunique() - 1, axis=1)

In [ ]:
addresses.head()

In [ ]:
# add address counts to the voter info, so that I can separate by age
count_addresses = addresses[['VOTER_ID', 'counts']]
voter_info = voter_info.merge(count_addresses, how='left', on=['VOTER_ID'], sort=False)

In [ ]:
voter_info.head()

In [ ]:
# dump into a csv
voter_info.to_csv('voter_info.csv', sep='\t', mode = 'w', index=False)

In [3]:
# neighborhoods
nb = pd.read_csv('voter_with_nbd2.csv')

In [ ]:
voterlatlong.shape

In [5]:
voterlatlong['Lat'] = voterlatlong['Lat'].apply(pd.to_numeric, downcast='float', errors='coerce')

In [6]:
voterlatlong['Long'] = voterlatlong['Long'].apply(pd.to_numeric, downcast='float', errors='coerce')

In [29]:
temp = nb[['addresses', 'NAME']]
temp = temp.drop_duplicates()

In [35]:
temp.head()

,addresses,NAME
0,3542 SE TAYLOR ST PORTLAND OR 97214,SUNNYSIDE-MULTNOMAH COUNTY
2,3105 SE HAWTHORNE BLVD APT A PORTLAND OR 97214,SUNNYSIDE-MULTNOMAH COUNTY
5,2825 SE HAWTHORNE BLVD PORTLAND OR 97214,SUNNYSIDE-MULTNOMAH COUNTY
7,4202 SE SALMON ST PORTLAND OR 97215,SUNNYSIDE-MULTNOMAH COUNTY
8,3120 SE STARK ST APT 2030 PORTLAND OR 97214,SUNNYSIDE-MULTNOMAH COUNTY


In [37]:
# join on the addresses to get neighborhoods
voterlatlong = voterlatlong.join(temp.set_index('addresses'), on='addresses')

In [171]:
# 1. separate by year, 2006 - 2016
# 2. separate by age
# 3. count neighborhoods
# we want for each neighborhood, a line plot of occupancy for age groups over time (years)

temp = voterlatlong[['VOTER_ID', 'dates', 'NAME']]
temp = temp.dropna()

years = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']
dictyear = dict.fromkeys(years)
for year in years:
    temp2 = temp[temp.dates.apply(lambda x: year in x)]
    temp2 = temp2.join(df.set_index('VOTER_ID'), on='VOTER_ID')
    temp2['curr_age'] = int(year) - temp2.year
    dictyear[year] = temp2
    # dump into a csv
    temp2.to_csv('{}.csv'.format(year), sep='\t', mode = 'w', index=False)

In [161]:
df = pd.read_csv('voter_info.csv', sep='\t')
df.head()
df = df.drop(['age2018', 'age2016', 'age2006'], axis=1)

In [210]:
# rows: neighborhood, columns: age group, counts
nb = pd.DataFrame(index=nb.NAME.unique())

In [191]:
nbcounts = temp.loc[(temp.curr_age>=20) & (temp.curr_age<=30) , 'NAME'].value_counts()
nbcounts = nbcounts.rename(year)

In [222]:
# 4 age groups
age_groups = [18, 26, 33, 40, 50, 150];
ages = np.ones([5, 2])
for i in range(0, len(age_groups) - 1):
    ages[i, :] = [age_groups[i], age_groups[i+1]]

In [249]:
idx = pd.MultiIndex.from_product([age_groups[:-1], nb.index], names=['ages', 'nb'])
df = pd.DataFrame(index=idx)
df = df.reset_index()

In [258]:
nbcounts = pd.DataFrame(nbcounts)
nbcounts['ages'] = 18

In [272]:
# nbcounts = nbcounts.reset_index()
nbcounts = nbcounts.rename(columns={'index':'nb'})
nbcounts.head()

,nb,2016,ages
0,CENTENNIAL,2377,18
1,HAZELWOOD,2364,18
2,NORTHWEST DISTRICT,1975,18
3,POWELLHURST-GILBERT,1873,18
4,RICHMOND,1843,18


In [275]:
df = pd.merge(df, nbcounts,  how='left', left_on=['ages','nb'], right_on = ['ages','nb'])

In [281]:
df.iloc[170:200]

,ages,nb,2016
170,18,MULT CO TROUTDALE AREA,NaN
171,18,COTTRELL,NaN
172,18,BORING,NaN
173,18,DAMASCUS,NaN
174,26,SUNNYSIDE-MULTNOMAH COUNTY,NaN
175,26,ROSE CITY PARK,NaN
176,26,GRANT PARK-MULTNOMAH COUNTY,NaN
177,26,BRENTWOOD/ DARLINGTON,NaN
178,26,NORTHWEST DISTRICT,NaN
179,26,DOWNTOWN,NaN


In [ ]:
# find people living in the same house
# for each address, count the number of occupants, record who was living there, if they were living at the same time
# or if they were living in different times (rental property)
voteraddresses = voterlatlong[['addresses', 'VOTER_ID', 'dates']]
dates = [c for c in addresses.columns if '-' in c]
date_dict = dict.fromkeys(dates)

# WORK FROM HERE, SAVE TO INDIVIDUAL DATES
for date in dates:
    temp = voteraddresses[voteraddresses['dates']==date]
    temp = temp[['addresses', 'VOTER_ID']]
    # number of occupants in the property for each date
    temp = temp.groupby('addresses', sort=False)
    temp2 = temp.VOTER_ID.nunique()
    temp2 = temp2.to_frame()
    temp2 = temp2.rename(columns={'VOTER_ID': 'ID_counts'})
    temp2 = temp2.join(temp.VOTER_ID.apply(list), rsuffix='s')
    # later remove the addresses with occupants > 10, they're communal places like Reed college, Lewis Clark, RV park, 
    # homeless shelter...
    date_dict[date] = temp2

In [ ]:
# dates_shared = pd.DataFrame({'dates': dates})
# dates_shared['four_or_more'] = np.repeat(np.nan, dates_shared.shape[0])

for date in dates:
    temp = date_dict[date]
    temp = temp[temp.ID_counts<20]
    dates_shared.loc[dates_shared.dates==date, 'four_or_more'] = temp.loc[temp.ID_counts>4, 'ID_counts'].shape[0]/temp.shape[0]

In [ ]:
plt.plot(dates_shared.four_or_more)
plt.xlabel(dates)
plt.show()

In [ ]:
cnames = voter_bday.columns
cnames = [name for name in cnames if '-' in name]
voter_bday.columns

In [ ]:
unique_addresses = pd.Series()

max_count = len(cnames)
fb = FloatProgress(min=0, max=max_count) # instantiate the bar
display(fb) # display the bar

for c in cnames:
    addresses = voter_bday[c].dropna()
    addresses = pd.Series(addresses.unique())
    unique_addresses = unique_addresses.append(addresses)

    # signal to increment the progress bar
    fb.value += 1
    
unique_addresses = unique_addresses.unique()

In [ ]:
np.savetxt('addresses_only.txt', unique_addresses, fmt='%s', newline='\n')